In [1]:

# ******************************************************************************
# EO-Lab Tutorial
# Query and Download of EnMAP Data from the EOC Geoservice
# ******************************************************************************
#
# This script demonstrates how to query and download EnMAP data from the EOC
# Geoservice STAC catalogue. The code is organized into several sections:
#   1. IMPORTS: Standard and third-party modules with explanations.
#   2. USER CONFIGURATION: Parameters and settings you can adjust.
#   3. HELPER FUNCTIONS: Utility functions for processing commands and geometry.
#   4. ENMAPDOWNLOADER CLASS: Contains the logic for searching, filtering, and downloading.
#   5. MAIN SCRIPT EXECUTION: Initiates the download process for enabled collections.
#
# Pre-requisites:
#   - Python 3.x
#   - Required libraries: os, re, requests, datetime, pystac_client, pathlib, math
#   - A valid session (cookie) is required for downloading files.
#
# ******************************************************************************
#
# ------------------------------------------------------------------------------
# Session Cookie Instructions:
#
# To download data from EOC Geoservice using curl, you must first obtain the session 
# cookie from the EOC Geoservice UMS. The easiest way is to log in to the 
# Geoservice EnMAP L2A Download page using your browser. When you log in 
# (choose "External Identity Providers EO-LAB" in the top right), open the Web 
# Developer Tools (e.g., press F12 in Firefox) and switch to the "Network" tab.
#
# Enter your credentials to log in. Look for a GET-request to 
# download.geoservice.dlr.de that contains the session cookie. Right-click that 
# request and select "Copy" -> "Copy as cURL". Use this copied cURL command in the 
# script (below) to download files.
#
# IMPORTANT:
#   - The example cURL command provided here is only a template.
#   - You must create your own cURL command with your session cookie.
#   - Session cookies expire over time; for long downloads, repeat the procedure to 
#     obtain a new session cookie.
#
# To download additional files, simply replace the URL in the cURL command.
# ------------------------------------------------------------------------------
#
# ---------------------------
#           IMPORTS
# ---------------------------
import os              # For operating system interactions (file paths, directories)
import re              # For regular expressions (pattern matching and extraction)
import requests        # For making HTTP requests (downloading files)
from datetime import datetime  # For date/time manipulation if needed
from pystac_client import Client  # For accessing and querying the STAC catalogue
from pathlib import Path  # For high-level file system path operations
from math import cos, radians  # For geographic calculations (used to compute bounding boxes)

# ---------------------------
#      USER CONFIGURATION
# ---------------------------

# CURL command simulating a browser request; used to extract necessary HTTP headers.
CURL_COMMAND = """
curl "https://download.geoservice.dlr.de/ENMAP/files/L2A/?ticket=ST-11848-nWxfdPAbhjmUoQdgPeS8uTr-gps-auth" -H "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:128.0) Gecko/20100101 Firefox/128.0" -H "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/png,image/svg+xml,*/*;q=0.8" -H "Accept-Language: de-DE,de;q=0.8,en-US;q=0.5,en;q=0.3" -H "Accept-Encoding: gzip, deflate, br, zstd" -H "DNT: 1" -H "Connection: keep-alive" -H "Cookie: session=Gpmbm3K-UAvKO0UmTO49OQ^|1739786884^|9zxCUa2Pz8OLhoe4zRQSYX515svNoPyKsLa93-7PevePOm7SkQBhBtKkWtOczQDe_v7f36KBsDIvQ9JIeJynkJntLlgpB3RKfsyqZV7fblipH2uMRJE3n4BDurIwMymM9KL4I8dEOruktKRpLCdgxyjurRXAQH80bK97bXq5Y2k^|cPfahrZ_q1kpCsKub_5j4QiiPlI" -H "Upgrade-Insecure-Requests: 1" -H "Sec-Fetch-Dest: document" -H "Sec-Fetch-Mode: navigate" -H "Sec-Fetch-Site: same-site" -H "Priority: u=0, i" """

# Extract base URL from CURL_COMMAND (for potential modifications)
CURL_PARTS = CURL_COMMAND.split(' ')
BASE_URL = CURL_PARTS[1][1:-1]

# Cloud cover filter: only process items with cloud cover below the specified maximum.
CLOUD_COVER_FILTER = {
    "enabled": True,         # Set to True to enable filtering.
    "max_coverage": 5.0        # Maximum allowed cloud cover percentage.
}

# Flag to print all properties (metadata) for each item.
PRINT_PROPERTIES = True

# Define which collections to download and the asset types for each.
DOWNLOADS = {
    # L2A Collection (uses lowercase asset names)
    "ENMAP_HSI_L2A": {
        "enabled": True,  # Enable this collection for processing.
        "assets": [
            "image",  # Main spectral image.
            # Uncomment additional asset types as needed:
            # "metadata", "vnir", "swir", "thumbnail", etc.
        ]
    },
    # L0 Quicklook Collection (uses UPPERCASE asset names)
    "ENMAP_HSI_L0_QL": {
        "enabled": False,  # Set to True to process this collection.
        "assets": [
            "THUMBNAIL",  # Quicklook thumbnail image.
            "OVERVIEW",   # Quicklook overview image.
            # Additional asset types can be added here.
        ]
    }
}

# Area of interest configuration:
# Option 1: Define a bounding box [west, south, east, north]
BBOX = [11.230259, 48.051808, 11.337891, 48.117059]  # Example: DLR Oberpfaffenhofen

# Option 2: Use a center coordinate with a specified box size (km)
USE_CENTER_COORD = True  # Set to True to use center coordinate instead of a bounding box.
CENTER_COORD = {
    "lat": 50.71868778231684,  # Center latitude.
    "lon": 7.158329088235492,  # Center longitude.
    "size_km": 3              # Size of the square area in kilometers.
}

# Time range for the search (format: YYYY-MM-DD)
START_DATE = "2024-01-01"  # Start date.
END_DATE = "2024-12-31"    # End date.

# Maximum number of items to process per collection (None means all found items)
MAX_ITEMS = None

# Download directory settings:
CUSTOM_DOWNLOAD_PATH = None  # e.g., "/home/user/my_enmap_data" or "C:/EnMAP_Data"
BASE_DIR = "EnMAP_downloads"  # Default directory if no custom path is provided.

# ---------------------------
#     HELPER FUNCTIONS
# ---------------------------
def clean_curl_command(curl_command):
    # -----------------------------------------------------------
    # Clean and format the provided CURL command.
    # - Removes extra newlines and spaces.
    # - Fixes cookie formatting (replaces '^|' with '|').
    # - Ensures the command is properly quoted.
    # -----------------------------------------------------------
    curl_command = ' '.join(curl_command.split())  # Remove extra whitespace.
    if 'Cookie:' in curl_command:
        cookie_start = curl_command.find('Cookie:')
        cookie_end = curl_command.find('" -H', cookie_start)
        if cookie_end == -1:
            cookie_end = curl_command.find('"', cookie_start + 15)
        cookie_part = curl_command[cookie_start:cookie_end]
        cleaned_cookie = cookie_part.replace('^|', '|')
        curl_command = curl_command[:cookie_start] + cleaned_cookie + curl_command[cookie_end:]
    if not curl_command.startswith('"""'):
        curl_command = '"""' + curl_command + '"""'
    return curl_command

def parse_curl_command(curl_command):
    # -----------------------------------------------------------
    # Extract headers from the provided CURL command.
    # Returns a dictionary mapping header names to their values.
    # -----------------------------------------------------------
    curl_command = clean_curl_command(curl_command)
    headers = {}
    header_pattern = r'-H\s*"([^:]+):\s*([^"]+)"'
    matches = re.findall(header_pattern, curl_command)
    for header, value in matches:
        headers[header] = value
    return headers

def create_bbox_from_center(lat, lon, size_km):
    # -----------------------------------------------------------
    # Create a bounding box from a center coordinate and box size.
    #
    # Parameters:
    #   lat     - Center latitude.
    #   lon     - Center longitude.
    #   size_km - Size of the box in kilometers.
    #
    # Returns:
    #   A list [west, south, east, north] representing the bounding box.
    # -----------------------------------------------------------
    km_per_degree_lat = 111.0  # Approximate kilometers per degree latitude.
    km_per_degree_lon = 111.0 * cos(radians(lat))  # Adjusted for the given latitude.
    lat_offset = (size_km / 2) / km_per_degree_lat
    lon_offset = (size_km / 2) / km_per_degree_lon
    west = lon - lon_offset
    east = lon + lon_offset
    south = lat - lat_offset
    north = lat + lat_offset
    return [west, south, east, north]

# ---------------------------
#    ENMAPDOWNLOADER CLASS
# ---------------------------
class EnmapDownloader:
    # -----------------------------------------------------------
    # Class to search, filter, and download EnMAP data items.
    #
    # Utilizes the STAC catalogue to locate items, applies filters
    # (e.g., cloud cover), downloads specified assets, and optionally
    # prints all item properties (metadata).
    # -----------------------------------------------------------
    def __init__(self):
        self.catalog = Client.open("https://geoservice.dlr.de/eoc/ogc/stac/v1/")
        self.headers = parse_curl_command(CURL_COMMAND)
        
    def setup_download_directory(self, collection_name):
        # -----------------------------------------------------------
        # Create the download directory if it does not exist.
        #
        # Parameters:
        #   collection_name - The folder name for the collection.
        #
        # Returns:
        #   Full path to the download directory.
        # -----------------------------------------------------------
        if CUSTOM_DOWNLOAD_PATH:
            base_path = os.path.expanduser(CUSTOM_DOWNLOAD_PATH)
            target_directory = os.path.join(base_path, collection_name)
        else:
            target_directory = os.path.join(BASE_DIR, collection_name)
        try:
            if not os.path.exists(target_directory):
                os.makedirs(target_directory)
                print(f"\nCreated directory at: {target_directory}")
            else:
                print(f"\nUsing existing directory at: {target_directory}")
        except Exception as e:
            print(f"Error creating directory: {str(e)}")
            print("Falling back to default directory")
            target_directory = os.path.join(BASE_DIR, collection_name)
            if not os.path.exists(target_directory):
                os.makedirs(target_directory)
        return target_directory

    def download_file(self, url, output_path):
        # -----------------------------------------------------------
        # Download a single file from the specified URL.
        #
        # Parameters:
        #   url         - URL of the file to download.
        #   output_path - Local path to save the downloaded file.
        #
        # Returns:
        #   True if the download is successful, False otherwise.
        # -----------------------------------------------------------
        if 'Referer' in self.headers:
            self.headers['Referer'] = os.path.dirname(url) + '/'
        try:
            response = requests.get(url, headers=self.headers, stream=True, allow_redirects=True)
            response.raise_for_status()
            total_size = int(response.headers.get('content-length', 0))
            with open(output_path, 'wb') as f:
                if total_size == 0:
                    f.write(response.content)
                else:
                    downloaded = 0
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                            downloaded += len(chunk)
                            progress = int(50 * downloaded / total_size)
                            print(f"\rProgress: [{'=' * progress}{' ' * (50 - progress)}] {downloaded}/{total_size} bytes", end='')
            print(f"\nSuccessfully downloaded: {output_path}")
            return True
        except Exception as e:
            print(f"Error downloading file: {str(e)}")
            if os.path.exists(output_path):
                os.remove(output_path)
            return False

    def search_and_download_collection(self, collection, asset_types):
        # -----------------------------------------------------------
        # Search for items in the specified collection, apply filters,
        # and download the defined asset types.
        #
        # If PRINT_PROPERTIES is True, prints all properties for each item.
        #
        # Parameters:
        #   collection  - Name of the collection.
        #   asset_types - List of asset types (keys) to download from each item.
        # -----------------------------------------------------------
        print(f"\nSearching for {collection} data...")
        search_params = {
            "collections": [collection],
            "bbox": create_bbox_from_center(CENTER_COORD["lat"],
                                              CENTER_COORD["lon"],
                                              CENTER_COORD["size_km"]) if USE_CENTER_COORD else BBOX
        }
        if START_DATE and END_DATE:
            search_params["datetime"] = f"{START_DATE}T00:00:00Z/{END_DATE}T23:59:59Z"
        try:
            search = self.catalog.search(**search_params)
            total_matches = search.matched()
            if total_matches == 0:
                print("No items found matching your criteria.")
                return
            print(f"Found {total_matches} items.")
            items = list(search.items())
            if CLOUD_COVER_FILTER["enabled"]:
                filtered_items = []
                print("\nFiltering by cloud cover...")
                for item in items:
                    cloud_cover = float(item.properties.get("eo:cloud_cover", 100.0))
                    if cloud_cover <= CLOUD_COVER_FILTER["max_coverage"]:
                        filtered_items.append(item)
                items = filtered_items
                print(f"After cloud cover filtering ({CLOUD_COVER_FILTER['max_coverage']}% max): {len(items)} items")
            items_to_process = min(MAX_ITEMS, len(items)) if MAX_ITEMS else len(items)
            print(f"Will download {items_to_process} items.")
            download_dir = self.setup_download_directory(collection.split('_')[-1])
            items_processed = 0
            for item in items:
                if MAX_ITEMS and items_processed >= MAX_ITEMS:
                    break
                print(f"\nProcessing item {items_processed + 1}/{items_to_process}")
                print(f"Cloud cover: {item.properties.get('eo:cloud_cover', 'N/A')}%")
                if PRINT_PROPERTIES:
                    print("Item properties:")
                    for key, value in item.properties.items():
                        print(f"  {key}: {value}")
                assets = item.get_assets()
                if items_processed == 0:
                    print("\nAvailable assets in first item:")
                    for asset_name in assets.keys():
                        print(f"- {asset_name}")
                    print()
                for asset_type in asset_types:
                    if asset_type in assets:
                        asset = assets[asset_type]
                        filename = os.path.basename(asset.href)
                        output_path = os.path.join(download_dir, filename)
                        print(f"\nDownloading {asset_type}: {filename}")
                        self.download_file(asset.href, output_path)
                    else:
                        print(f"\nAsset type {asset_type} not found in item")
                items_processed += 1
            print(f"\nDownload complete for {collection}!")
        except Exception as e:
            print(f"Error during search and download: {str(e)}")

    def download_all(self):
        # -----------------------------------------------------------
        # Iterate over all defined collections in DOWNLOADS.
        # For each enabled collection, process and download the specified assets.
        # -----------------------------------------------------------
        for collection, config in DOWNLOADS.items():
            if config["enabled"]:
                print(f"\n{'=' * 50}")
                print(f"Processing collection: {collection}")
                print(f"{'=' * 50}")
                self.search_and_download_collection(collection, config["assets"])

# ---------------------------
#      MAIN SCRIPT EXECUTION
# ---------------------------
if __name__ == "__main__":
    downloader = EnmapDownloader()
    downloader.download_all()



Processing collection: ENMAP_HSI_L2A

Searching for ENMAP_HSI_L2A data...
Found 7 items.

Filtering by cloud cover...
After cloud cover filtering (5.0% max): 2 items
Will download 2 items.

Using existing directory at: EnMAP_downloads/L2A

Processing item 1/2
Cloud cover: 2%
Item properties:
  version: 01.04.02
  created: 2024-07-05T11:16:53.923+00:00
  updated: 2024-07-05T11:16:53.923+00:00
  datetime: 2024-06-26T11:11:39Z
  start_datetime: 2024-06-26T11:11:39.641792Z
  end_datetime: 2024-06-26T11:11:44.176515Z
  platform: enmap
  proj:epsg: None
  sat:relative_orbit: 12040
  sat:orbit_state: DESCENDING
  eo:cloud_cover: 2
  eo:snow_cover: 0
  view:sun_azimuth: 348.49244999999996
  view:sun_elevation: 62.246176
  processing:facility: NZ
  processing:level: L2A
  eop:sensorType: OPTICAL
  card4l:eastern_geometric_accuracy: 6.0
  card4l:geometric_accuracy_radial_rmse: 10.0
  card4l:northern_geometric_accuracy: 8.0
  card4l:specification: SR
  card4l:specification_version: 5.0
  enmap:S